## The forward and backward passes
L14 2022p2

See also [Simple Neural Net Backward Pass - Deriving the math of the backward pass for a simple neural net](https://nasheqlbrm.github.io/blog/posts/2021-11-13-backward-pass.html)

### Preliminaries: imports and data

In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

Get the MNIST data as tensors.

In [2]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations version

### Basic architecture
Lets start by defining a few variables: `n` is the number of training examples, `m` is the number of pixels, `c` is the number of possible values of our digits.
Here there are 50,000 training samples, 784 pixels and 10 possible outputs.  

In [3]:
n,m = x_train.shape
c = y_train.max()+1   # number of values
n,m,c

(50000, 784, tensor(10))

We decide (ahead of time) how many "line segments" to add up. 
The number in a layer is the *number of hidden nodes or activations*, `nh`.
Lets arbitrarily decide `nh=50`.

In [4]:
# num hidden
nh = 50

To create lots of "lines", which we are then going to truncate at zero we do a matrix multiplication. 
Later we're going to have 50000x784 to multiply by a 784x10.
But to simplify our starting point, lets we give layer 2 just 1 output, so we can use MSE.

In [5]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)
w1.shape, b1.shape, w2.shape, b2.shape

(torch.Size([784, 50]), torch.Size([50]), torch.Size([50, 1]), torch.Size([1]))

Also lets use the smaller `x_valid` matrix.

In [6]:
x_valid.shape

torch.Size([10000, 784])

A simple linear layer `lin`.

In [7]:
def lin(x, w, b): return x@w + b

We call it and should return a `[10000,50]` matrix.

In [8]:
t = lin(x_valid, w1, b1)
t.shape, t

(torch.Size([10000, 50]),
 tensor([[ -0.09,  11.87, -11.39,  ...,   5.48,   2.14,  15.30],
         [  5.38,  10.21, -14.49,  ...,   0.88,   0.08,  20.23],
         [  3.31,   0.12,   3.10,  ...,  16.89,  -6.05,  24.74],
         ...,
         [  4.01,  10.35, -11.25,  ...,   0.23,  -5.30,  18.28],
         [ 10.62,  -4.27,  10.72,  ...,  -2.87,  -2.87,  18.23],
         [  2.84,  -0.22,   1.43,  ...,  -3.91,   5.75,   2.12]]))

In [9]:
def relu(x): return x.clamp_min(0.)

t = relu(t); t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

Now lets define our basic MLP from scratch.  It would be:
```python
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)
```

In [10]:
#Compressed model
def model(xb):
    return lin(relu(lin(xb, w1, b1)), w2, b2)

In [11]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Loss function: MSE

NB: `mse` is not a suitable loss function for multi-class classification; We'll use `mse` for now to keep things simple.

In [12]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

If we just substract, broadcasting creates a problem as it creates a huge matrix...

In [13]:
(res-y_valid).shape   

torch.Size([10000, 10000])

We need to get rid of that trailing axis of `res` (,1), in order to use `mse`.
We either use the single column of `res` or we `squeeze` `res`.

In [14]:
res[:,0].shape  # either use the single column

torch.Size([10000])

In [15]:
res.squeeze().shape  # Or use squeeze

torch.Size([10000])

In [16]:
(res[:,0]-y_valid).shape

torch.Size([10000])

To use MSE we need the values of the labels `y` to be floats, but they are integers.

In [17]:
y_train[2], y_train[2].float()

(tensor(4), tensor(4.))

Let's get our training and validation into floats because we're using MSE.   

In [18]:
y_train,y_valid = y_train.float(),y_valid.float()

Let's calculate our predictions for the training set, `x_train`, which is `[50000,1]`. 

In [19]:
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

We define an `mse` function that does the subtraction of the passed arguments, squares it `.pow(2)` and takes the mean.

And apply this `mse` loss function to the predictions of our model `preds`, and the labels of the training set, `y_train`.

In [20]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

mse(preds, y_train)

tensor(4308.76)

### Gradients and backward pass

See also [Simple Neural Net Backward Pass - Deriving the math of the backward pass for a simple neural net.](https://nasheqlbrm.github.io/blog/posts/2021-11-13-backward-pass.html)

We can use [SymPy](https://www.sympy.org/en/index.html) a Python library for symbolic mathematics. 
Wolfram Alpha does something similar. 
With SimPy we can do it inside a notebook and include it in prose.<br>
For example, we define two symbols `x` and `y`, 
we tell it to differentiate $x^3$ with respect to `x`, and SimPy will answer $3x^2$.

In [21]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**3, x)

3*x**2

To differentiate $3x^2 + 9$ with respect to $x$:  
 

In [22]:
diff(3*x**2+9, x)

6*x

`lin_grad` computes the gradient of a linear layer.
Per the chain rule, we need: the input `inp`, output `out`, weights `w`, and the biases `b` of the layer.
We will store the gradients of our input in `inp.g`, 
which is `out.g @ w.t()` the gradients of `out` with respect to the input times the weights.
A matrix multiplier is a whole bunch of linear functions, so each one slope is just its weight.  
But we have to multiply it by the gradient of the outputs because of the chain rule.  
The gradient of the outputs with respect to the weights, `w.g`, is the input times the output summed up.
Every input weights has to be multiplied by the outputs, that's why we have to  do an `unsqueeze(-1)`. 
The  derivatives of the bias, `b.g`, is the gradients of the output added together
because the bias is just a constant value. 

In [23]:
def lin_grad(inp, out, w, b):  #the gradient of a linear layer
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    #import pdb; pdb.set_trace()
    i, o = inp.unsqueeze(-1) , out.g.unsqueeze(1)
    w.g = (i * o).sum(0)
    b.g = out.g.sum(0)

The **forward** pass is where we calculate the `loss`, which is diff (the output `out` 
of the neural net minus our target) squared and then take the mean. 
`out` is the output of the 2nd linear layer `l2`. 
The input to `l2` is the ReLU, and the ReLU's input is the first layer, `l1`. 
We take the input,`inp` put it through a linear layer `l1`, through a ReLU, 
through a linear layer `l2` and calculate the MSE.

In the **backward** pass, we store the gradients of each layer (e.g., loss with respect to inputs), in the layer itself.
We define a new attribute, `g`. 
We define a new attribute called out.g, to contain the gradients.
In `out.g = 2.*diff[:,None] / inp.shape[0]`
the derivative is two times the difference because we've got difference squared.  
We took the mean when computing the loss, so we have to do the same thing here, i.e., divided by the input shape.
Now we need to multiply by the gradients of the previous layer, `l2`.
To compute the gradients of a linear layer we use `lin_grad` above.
Per the chain rule, we need: the weights `w2` and the biases `b2` of the `l2` layer, 
and also the input `l2` and the output `out` from the linear layer.

In [24]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]  # the gradients saved
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [25]:
forward_and_backward(x_train, y_train)

Lets save all the gradients for `w1,w2,b1,b2,x_train`,
in a list `grads` (for testing against the Pytorch equivalents later).

In [26]:
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = map(get_grad, chks)

Lets save all the gradients for `w12,w22,b12,b22,xt2`, the equivalents to before in a list `ptgrads`.

In [27]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = map(mkgrad, chks)


We just run it all through PyTorch and check that their derivatives `ptgrads` are the same.

In [28]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

We cheat a little bit and use PyTorch autograd to check our results.

In [29]:
loss = forward(xt2, y_train)
loss.backward()

We test the calculated derivatives by comparing them with the same derivatives calculated by PyTorch.

In [30]:
for a,b in zip(grads, ptgrads): test_close(a.grad, b, eps=0.01)

## Refactor model

We can refactor and simplify by using classes and invoking them as functions.
Lets illustrate by a class just to print hello.
We create an instance of that class and then we can call it as if it was a function.
In Python we can change how a class behaves, make it work like a function, by defining `__call__`. 
It is a syntactic sugar to treat a class as if it's a function without any method  at all. 
We can still do it the method way, but why do that?

In [31]:
class A:
    def __call__(self, x): print(f'hi {x}')
    
a = A()
a("j")

hi j


### Layers as classes

Lets simplify and define classes for `ReLU` and for the linear function `Lin`. 

Lets define the ReLU class and add `__call__` so we can treat it as a function. 
Note that the backward pass has to know about the intermediate calculations
because of the chain rule, and because of how the derivatives are calculated.
We need to store each of the layer intermediate calculations.
The ReLU class stores its output and its input, so when we call the `backward` method, 
we know how to calculate that. 
We set the inputs gradient, `self.inp.g` by the chain rule the product of 2 derivatives,
`(self.inp>0).float() * self.out.g`.

In [32]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

Lets do the same thing for a linear layer class `Lin`. 
A linear layer needs some additional state to be passed: weights and  biases. (ReLU doesn't). 
We indicate its weights `w` and biases `b`, and store them.
When we `__call__` it on the forward pass we store the input `inp`, then 
compute the output, store it in `self.out`, and `return` it. <br>
For the backward pass, the input gradients we calculate as before. 
`.t()` is the same as `T` is as a property: the transpose.  
We calculate the gradients of `inp, w, b` and store them in the appropriate `.g` places.

Below to compute the derivation of the gradients in `backward()`:
```python      
dJ_dZ = self.out.g  # Gradient of loss with respect to the output     

self.w.g = dJ_dW = self.inp.t() @ dJ_dZ # Gradient of loss with respect to w_j     

self.b.g = dJ_db = dJ_dZ.sum(0)   #Gradient of loss with respect to the bias b

self.inp.g = dJ_dX = dJ_dZ @ self.w.t() # Gradient of loss with respect to X

```

In [33]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()  # See Gradient of loss with respect to X   
        self.w.g = self.inp.t() @ self.out.g # See Gradient of loss with respect to w_j
        self.b.g = self.out.g.sum(0) # See Gradient of loss with respect to the bias b

The backward function of the Mse class below computes an estimate of how the loss function changes as the input activations change.

For MSE we do the same thing we calculate it and store it in `self.out`. 
MSE needs input and target, so we store them. 
In the backward pass we can calculate its gradient of the input as being two times the difference. 
For the backward we compute it as:
```python
N = self.targ.shape[0] ;  A = self.inp ; Y = self.targ
self.inp.g = dJ_dA = (2./N) * (A.squeeze() - Y).unsqueeze(-1)
```

In [34]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

Th model is easier to define as a list of layers, `[Lin(w1,b1), Relu(), Lin(w2,b2)]`. 
We store in `self.loss` an instance of `Mse()`.
NB: These are not calls, just instances of the classes (`Lin, Relu, Mse`) being stored, 
so when we call the model we pass it our inputs and our target. 
In `__call__` we go through each layer, set `x` equal to the result of calling that layer, 
and then pass that to the `loss`. 
<br>
Notice that we don't have two separate functions, the loss function being applied to a separate neural net.
Rather we integrated the loss function into the model, i.e., the loss is calculated inside the model.
That is different, neither better nor worse than having it separately.
HuggingFace stuff does it this way, it puts the `loss` inside the `forward`.
Fastai and other libraries does it separately, i.e., loss is a whole separate function, 
and the model only returns the result of putting it through the layers.
For this model the loss function is inside the model.
<br>
For backward, `self.loss` is the `Mse()` object. 
So that's going to call `loss.backward()`, and it's stored when it was called here.  
It stores  the inputs, the targets, the outputs, so it can calculate the `backward()`.
Then we go through each layer in reverse, the **back propagation**, `backwards` `reversed`, 
calling `l.backward()` on each one. 

In [35]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

Q: if we just return the `loss` above in the `__call__`, how do you get predictions?  
A: HuggingFace models return not just the `loss`, but a dictionary, 
i.e., `dict(loss=..., preds=...)`, something like that. 

Now we can calculate the `Model`, calculate the `loss`, and call `backward`. 

In [36]:
model = Model(w1, b1, w2, b2)

loss = model(x_train, y_train)

model.backward()

And then we can check that each of the gradients that we stored earlier are equal to each of our new gradients.

In [37]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

Repeated code,  e.g., `self.inp=inp`, etc., is a sign that we can refactor things.  
Lets define a new class `Module()` to do those things that are repeated. 
It's going to store the inputs, call `self.forward` to create  the `self.out`, and then return it. 
There's going to be a `forward` which in this class it doesn't do anything,
as the purpose of this Module is to be inherited. 
When we call backward, it's going to call `self.bwd` passing in 2 arguments:
(1) `self.out` because all `backwards()` wanted to get `self.out` because of the chain rule,
and (2)the arguments that we stored earlier. 
`*` in a signature (e.g., `def __call__(self, *args)`
means take all of the arguments, regardless of number, and put them into a list. 
When we call a function using `*`, e.g., `self.bwd(self.out, *self.args)`
it says take this list and expand them into separate arguments,
and pass them (e.g. to `self.bwd`)  each one separately.

In [38]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [39]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [40]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [41]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

There are often opportunities to manually speed-up by defining custom Pytorch autograd functions.
For example, in `Mse` a calculation  `inp.squeeze() - targ` is being done twice.
At the cost of some memory, we could instead store that calculation as, e.g., `self.diff`.
And at the cost of that memory, we could now remove this redundant calculation 
because we've done it once before already and  stored it and just use it directly. 
This is something that you can often do in neural nets, 
a compromise between memory use and then the computational speedup of not having to recalculate it.  

Now we can call it in the same way, create the model, passing in all of those layers. 
The model  hasn't changed at this point. 
The definition was up here, we just pass in the weights for the layers,
calculate the loss, call backward, and it's the same.

In [42]:
model = Model(w1, b1, w2, b2)

loss = model(x_train, y_train)

model.backward()

In [43]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd
PyTorch has all this, and since we've reimplemented it, we can use PyTorch's version, `nn.Module`.  

In [44]:
from torch import nn
import torch.nn.functional as F

To define a `Linear` layer we inherit from `nn.Module`.
Here rather than passing in the already randomized weights, we generate the random weights and the zeroed biases.
We define `forward` but we don't need to define `backward`, as PyTorch 
knows all the derivatives and the chain rule, it will do it. 

In [45]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

Let's define a `Model` class that uses `nn.Module`, it's the same as before,
but now we use PyTorch's `F.mse_loss()`.<br>
NB: We need the extra axis in `targ[:,None]` as we saw the problem if we don't have it. 

In [46]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

We create the model, call backward.
We stored our gradients in `.g`, PyTorch stores them in `.grad`.
The same values.

In [47]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [48]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])

To summarize, we've created from scratch:
* a matrix multiplication
* linear layers
* a complete backprop system of modules 

We can now calculate both the forward pass and the  backward pass for linear layers and values so
we can create a multilayer perceptron, and we can train a model.